<a href="https://colab.research.google.com/github/junwoochoi33/LLMPractice/blob/main/chapter_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 토큰화 코드

In [2]:
# 띄어쓰기 단위로 분리
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids:  [0, 1, 2, 3, 4]


## 2. 토큰 아이디에서 벡터로 변환

In [3]:
import torch
import torch.nn as nn

embedding_dim = 16
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

input_embeddings = embed_layer(torch.tensor(input_ids)) # (5, 16)
input_embeddings = input_embeddings.unsqueeze(0) # (1, 5, 16)
# input_embeddings.shape
input_embeddings

tensor([[[ 0.6579,  0.7739,  0.2675,  0.0124, -0.5073, -0.8846,  0.2332,
           1.6807,  0.9881, -0.5445,  0.6446, -0.7269, -1.5648,  0.5873,
           0.5666,  0.7179],
         [-0.8121, -1.3139,  0.1626,  0.7064, -2.7119,  0.8493,  0.8502,
          -0.3744,  0.6835,  0.3048,  1.2164,  1.0019, -0.9343,  0.2284,
           0.2863, -1.4321],
         [ 1.2773,  0.4492, -0.3486,  0.3783, -0.2183,  0.3061,  0.2332,
           0.5297,  0.3130, -0.4391, -0.0204, -2.5164,  1.3610,  0.0776,
          -1.4333, -0.1875],
         [-1.2219, -1.1149, -0.4634, -0.1875, -1.5990,  0.0507,  0.1052,
          -1.1133, -0.5529,  0.2957, -0.0233,  0.6669, -0.9267,  1.4350,
           0.5256, -0.7110],
         [ 1.3427, -0.4236,  0.6874,  0.2332, -0.7898, -1.4387,  0.1010,
          -0.8769, -0.2246,  0.0749, -0.1242, -2.3977, -0.2805,  1.7240,
          -2.1118, -1.1598]]], grad_fn=<UnsqueezeBackward0>)

## 3. 절대적 위치 인코딩

In [4]:
embedding_dim = 16 # (예: "사과" → [0.1, -0.3, ..., 0.5])
max_position = 12 # 문장은 최대 12단어까지 있다고 생각

# 토큰 임베딩 층 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim) # 단어들을 숫자 벡터로 바꿔주는 사전 (예: "사과"는 [0.1, 0.2, ..., 0.9])
# 위치 인코딩 층 생성
position_embed_layer = nn.Embedding(max_position, embedding_dim) # 단어가 몇 번째에 있는지, 그 위치도 벡터로 변경 (예: 첫 번째 단어는 [0.05, 0.02, ..., 0.1])

position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0) # (예: 단어가 5개라면 → 위치 번호는 [0, 1, 2, 3, 4])
position_encodings = position_embed_layer(position_ids) # 위치 번호를 숫자 벡터로 변경
token_embeddings = embed_layer(torch.tensor(input_ids)) # 진짜 단어들을 숫자 벡터로
token_embeddings = token_embeddings.unsqueeze(0) # 묶어서 하나의 문장(배치)로
# 토큰 임베딩과 위치 인코딩을 더해 최종 입력 임베딩 생성
input_embeddings = token_embeddings + position_encodings # 단어 정보 + 위치 정보
input_embeddings.shape # 문장 1개, 단어 5개, 숫자 16개짜리 벡터

torch.Size([1, 5, 16])

## 4. 쿼리, 키, 값 벡터를 만드는 nn.Linear 층

In [5]:
head_dim = 16

# 쿼리, 키, 값을 계산하기 위한 변환
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)
# 변환 수행
querys = weight_q(input_embeddings) # (1, 5, 16)
keys = weight_k(input_embeddings) # (1, 5, 16)
values = weight_v(input_embeddings) # (1, 5, 16)

## 5. 스케일 점곱 방식의 어텐션

In [6]:
from math import sqrt
import torch.nn.functional as F

def compute_attention(querys, keys, values, is_causal=False):
	dim_k = querys.size(-1) # 16
	scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k)
	weights = F.softmax(scores, dim=-1)
	return weights @ values

## 6. 어텐션 연산의 입력과 출력

In [7]:
print("원본 입력 형태: ", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("어텐션 적용 후 형태: ", after_attention_embeddings.shape)
# 원본 입력 형태:  torch.Size([1, 5, 16])
# 어텐션 적용 후 형태:  torch.Size([1, 5, 16])
after_attention_embeddings

원본 입력 형태:  torch.Size([1, 5, 16])
어텐션 적용 후 형태:  torch.Size([1, 5, 16])


tensor([[[-0.0153, -0.2035, -0.4220, -0.0793, -0.4480, -0.6191, -0.2218,
           0.3179,  0.2985,  0.0858,  0.0549, -0.3632,  0.5996, -0.1475,
          -0.3218, -0.2537],
         [ 0.3496, -0.7060, -0.2552,  0.1710, -0.2656, -0.5021,  0.0790,
           0.2876,  0.2558,  0.3944,  0.2773, -0.6310,  0.5420,  0.1765,
          -0.3556, -0.2713],
         [ 0.2573, -0.4879, -0.2974, -0.0011, -0.3615, -0.4739, -0.0488,
           0.2732,  0.3677,  0.2641,  0.2254, -0.5755,  0.5497,  0.0602,
          -0.4476, -0.1307],
         [ 0.0402, -0.3269, -0.3870,  0.1027, -0.4164, -0.6663, -0.0971,
           0.3332,  0.1425,  0.1342,  0.1187, -0.3641,  0.5396, -0.0361,
          -0.2518, -0.4468],
         [ 0.2968, -0.7680, -0.3206,  0.0824, -0.1973, -0.5305,  0.0149,
           0.3916,  0.3296,  0.3971,  0.2235, -0.4591,  0.6937,  0.0932,
          -0.2164, -0.2621]]], grad_fn=<UnsafeViewBackward0>)

## 7. 어텐션 연산을 수행하는 AttentionHead 클래스

In [9]:
class AttentionHead(nn.Module):
  def __init__(self, token_embed_dim, head_dim, is_causal=False):
    super().__init__()
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, head_dim) # 쿼리 벡터 생성을 위한 선형 층
    self.weight_k = nn.Linear(token_embed_dim, head_dim) # 키 벡터 생성을 위한 선형 층
    self.weight_v = nn.Linear(token_embed_dim, head_dim) # 값 벡터 생성을 위한 선형 층

  def forward(self, querys, keys, values):
    outputs = compute_attention(
        self.weight_q(querys),  # 쿼리 벡터
        self.weight_k(keys),    # 키 벡터
        self.weight_v(values),  # 값 벡터
        is_causal=self.is_causal
    )
    return outputs

attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)

## 8. 멀티 헤드 어텐션 구현

In [10]:
class MultiheadAttention(nn.Module):
  def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
    super().__init__()
    self.n_head = n_head
    self.is_causal = is_causal
    self.weight_q = nn.Linear(token_embed_dim, d_model)
    self.weight_k = nn.Linear(token_embed_dim, d_model)
    self.weight_v = nn.Linear(token_embed_dim, d_model)
    self.concat_linear = nn.Linear(d_model, d_model)

  def forward(self, querys, keys, values):
    B, T, C = querys.size()
    querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    attention = compute_attention(querys, keys, values, self.is_causal)
    output = attention.transpose(1, 2).contiguous().view(B, T, C)
    output = self.concat_linear(output)
    return output

n_head = 4
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape

torch.Size([1, 5, 16])

## 9. 층 정규화 코드

In [11]:
norm = nn.LayerNorm(embedding_dim)
norm_x = norm(input_embeddings)
norm_x.shape # torch.Size([1, 5, 16])

norm_x.mean(dim=-1).data, norm_x.std(dim=-1).data

# (tensor([[ 2.2352e-08, -1.1176e-08, -7.4506e-09, -3.9116e-08, -1.8626e-08]]),
#  tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

(tensor([[ 1.4901e-08,  7.4506e-09, -2.9802e-08,  2.9802e-08, -7.4506e-09]]),
 tensor([[1.0328, 1.0328, 1.0328, 1.0328, 1.0328]]))

## 10. 피드 포워드 층 코드

In [12]:
class PreLayerNormFeedForward(nn.Module):
  def __init__(self, d_model, dim_feedforward, dropout):
    super().__init__()
    self.linear1 = nn.Linear(d_model, dim_feedforward) # 선형 층 1
    self.linear2 = nn.Linear(dim_feedforward, d_model) # 선형 층 2
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃 층 1
    self.dropout2 = nn.Dropout(dropout) # 드랍아웃 층 2
    self.activation = nn.GELU() # 활성 함수
    self.norm = nn.LayerNorm(d_model) # 층 정규화

  def forward(self, src):
    x = self.norm(src)
    x = x + self.linear2(self.dropout1(self.activation(self.linear1(x))))
    x = self.dropout2(x)
    return x

## 11. 인코드 층

In [13]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward, dropout):
    super().__init__()
    self.attn = MultiheadAttention(d_model, d_model, nhead) # 멀티 헤드 어텐션 클래스
    self.norm1 = nn.LayerNorm(d_model) # 층 정규화
    self.dropout1 = nn.Dropout(dropout) # 드랍아웃
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout) # 피드포워드

  def forward(self, src):
    norm_x = self.norm1(src)
    attn_output = self.attn(norm_x, norm_x, norm_x)
    x = src + self.dropout1(attn_output) # 잔차 연결

    # 피드 포워드
    x = self.feed_forward(x)
    return x

## 12. 인코더 구현

In [15]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerEncoder(nn.Module):
  def __init__(self, encoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(encoder_layer, num_layers)
    self.num_layers = num_layers
    self.norm = norm

  def forward(self, src):
    output = src
    for mod in self.layers:
        output = mod(output)
    return output

## 13. 디코더에서 어텐션 연산(마스크 어텐션)

In [16]:
def compute_attention(querys, keys, values, is_causal=False):
	dim_k = querys.size(-1) # 16
	scores = querys @ keys.transpose(-2, -1) / sqrt(dim_k) # (1, 5, 5)
	if is_causal:
		query_length = querys.size(2)
		key_length = keys.size(2)
		temp_mask = torch.ones(query_length, key_length, dtype=torch.bool).tril(diagonal=0)
		scores = scores.masked_fill(temp_mask == False, float("-inf"))
	weights = F.softmax(scores, dim=-1) # (1, 5, 5)
	return weights @ values # (1, 5, 16)

## 14. 크로스 어텐션이 포함된 디코더 층

In [ ]:
class TransformerDecoderLayer(nn.Module):
  def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1):
    super().__init__()
    self.self_attn = MultiheadAttention(d_model, d_model, nhead)
    self.multihead_attn = MultiheadAttention(d_model, d_model, nhead)
    self.feed_forward = PreLayerNormFeedForward(d_model, dim_feedforward, dropout)

    self.norm1 = nn.LayerNorm(d_model)
    self.norm2 = nn.LayerNorm(d_model)
    self.dropout1 = nn.Dropout(dropout)
    self.dropout2 = nn.Dropout(dropout)

  def forward(self, tgt, encoder_output, is_causal=True):
    # 셀프 어텐션 연산
    x = self.norm1(tgt)
    x = x + self.dropout1(self.self_attn(x, x, x, is_causal=is_causal))
    # 크로스 어텐션 연산
    x = self.norm2(x)
    x = x + self.dropout2(self.multihead_attn(x, encoder_output, encoder_output))
    # 피드 포워드 연산
    x = self.feed_forward(x)
    return x

## 15. 디코더 구현

In [17]:
import copy
def get_clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

class TransformerDecoder(nn.Module):
  def __init__(self, decoder_layer, num_layers):
    super().__init__()
    self.layers = get_clones(decoder_layer, num_layers)
    self.num_layers = num_layers

  def forward(self, tgt, src):
    output = tgt
    for mod in self.layers:
        output = mod(output, src)
    return output